In [136]:
# Script to re-render all equations created by matlab to improve quality
import os
import re
import subprocess
from bs4 import BeautifulSoup

In [137]:
from PIL import Image

def sizeOfPNG(folder_path, fileName):
    image_path = fileName # os.path.join(folder_path, fileName)
    
    # Open the image using PIL
    img = Image.open(image_path)
    
    # Get the size (width and height) of the image
    width, height = img.size
    
    return width, height

In [138]:
import shutil

def moveFiles(folder_a,folder_b):
    for filename in os.listdir(folder_a):
        if filename.endswith(".png"):
            source_path = os.path.join(folder_a, filename)
            destination_path = os.path.join(folder_b, filename)
            shutil.move(source_path, destination_path)

def delete_aux_log_pdf_files(directory):
    # List all files in the directory
    file_list = os.listdir(directory)

    # Iterate through the files and delete .aux, .log, and .pdf files
    for filename in file_list:
        if filename.endswith(".aux") or filename.endswith(".log") or filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            os.remove(file_path)

def delete_nonPNGs_inFolder(folder_path):
    # List all files in the folder
    file_list = os.listdir(folder_path)

# Iterate through the files and delete those that are not .png files
    for file_name in file_list:
        if not file_name.endswith('.png'):
            file_path = os.path.join(folder_path, file_name)
            os.remove(file_path)
            print(f"Deleted: {file_name}")

In [139]:
# Define a function to create .tex files from equations in the HTML
def create_tex_files(input_html, output_directory,scale):
    with open(input_html, 'r', encoding='utf-8') as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    #equations = soup.find_all('img', {'alt'})

    img_tags = soup.find_all('img')
    
    for i, img_tag in enumerate(img_tags):
        # Check if the img tag exists and has an alt attribute with LaTeX content
        if img_tag and 'alt' in img_tag.attrs and img_tag['alt'].startswith('$') and img_tag['alt'].endswith('$'):
            latex_content = img_tag['alt']
              
            src_text = img_tag.get('src')
            
            latex_content = latex_content.replace('$', '')
            tex_code = f"\\documentclass[border={{0pt 0pt 0pt 1pt}}]{{standalone}}\n\\usepackage{{amsmath}}\n\\begin{{document}}\n${latex_content}$\n\\end{{document}}"

            # Create Tex document
            EqName = img_tag['src'].replace('.png','')           
            tex_filename = os.path.join(output_directory, f"{EqName}.tex")

            with open(tex_filename, 'w', encoding='utf-8') as tex_file:
                tex_file.write(tex_code)


            if 'Rescaled' not in src_text:    # rename attribute in html file

                img_tag['src'] = src_text.replace('.png', '-Rescaled.png')     


            # Adjust the width of the rescaled figure to match the original
            # Get size of PNGfile
            width, height = sizeOfPNG('', src_text)
            print(width)
            img_tag['style'] = f"width: {width*scale}px; height: auto;"
            

    with open('tmp.html', 'w', encoding='utf-8') as input_html:
        input_html.write(str(soup))
    
        


In [140]:
# Define a function to compile .tex files into PNG images
def compile_tex_to_png(input_directory, output_directory,current_directory,scale,dpi):
    for tex_file in os.listdir(input_directory):
        if tex_file.endswith(".tex"):

            base_name = os.path.splitext(tex_file)[0]

            # Get size of PNGfile
            width, height = sizeOfPNG(current_directory, f"{base_name}.png")

            if 'Rescaled' not in base_name: 
                output_png = os.path.join(output_directory, f"{base_name}-Rescaled.png")
                # delete old image
                os.remove(f"{base_name}.png")
            else:
                output_png = os.path.join(output_directory, f"{base_name}.png")

            # Compile the LaTeX file to PDF
            subprocess.run(["pdflatex", "-interaction=nonstopmode", "-output-directory", output_directory, os.path.join(input_directory, tex_file)])
                    
            width = width * scale
            height = height * scale

            '''
            # Convert the PDF to PNG
            subprocess.run(["pdftoppm", 
                            "-png",
                            f"{output_directory}/{base_name}.pdf", 
                            f"{output_directory}/{base_name}","-r", str(dpi), "-scale-to-x", f"{int(width)}","-scale-to-y", f"{int(height)}"
                              ])
            
            subprocess.run(["pdf2svg", 
                            f"{output_directory}/{base_name}.pdf", 
                            f"{output_directory}/{base_name}.svg"
                              ])
            # Rename the output file to match the original tex_file name
            os.rename(f"{output_directory}/{base_name}-1.png", output_png)                  
            '''
            
            pdf_to_png(f"{output_directory}/{base_name}.pdf", f"{output_directory}/{base_name}-Rescaled",width,dpi)

from pdf2image import convert_from_path
from PIL import Image


def pdf_to_png(input_pdf, output_png,width,dpis):
    images = convert_from_path(input_pdf, dpi=dpis,size=(20*width, None))  # Adjust DPI as needed
    for i, image in enumerate(images):

        # we only have one image/page per pdf
        image.save(f"{output_png}.png", "PNG")

In [141]:

# Filter the list to include only HTML files
file_list = os.listdir('.')
html_files = [file for file in file_list if file.endswith(".html")]

# Print the list of HTML files
for input_html_file in html_files:
    print(input_html_file)
    equation_directory = "equations"  # Directory with .tex files
    output_image_directory = "output_images"  # Directory with PNG images


    os.makedirs(equation_directory, exist_ok=True)
    os.makedirs(output_image_directory, exist_ok=True)

    scale = 1.2
    dpis = 300

    create_tex_files(input_html_file, equation_directory,scale)
    compile_tex_to_png(equation_directory, output_image_directory,'',scale,dpis)

    os.rename("tmp.html", input_html_file)
    moveFiles('output_images','')
    shutil.rmtree(equation_directory)
    shutil.rmtree(output_image_directory)

Bernoulli Beam.html
52
7
223
205
49
This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./equations/BernoulliBeamBaseDoc_eq07061455713171635040.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/local/texlive/2023basic/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2022/10/10 v1.3b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2023basic/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/local/texlive/2023basic/texmf-dist/tex/generic/iftex/ifluatex.sty
(/usr/local/texlive/2023basic/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/local/texlive/2023basic/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2023basic/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2023basic/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2023basic/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texli